In [3]:
import numpy as np
import urllib
import scipy.optimize
import random
from sklearn.metrics import mean_squared_error

In [4]:
def parseData(fname):
  for l in urllib.urlopen(fname):
    yield eval(l)

In [5]:
print "Reading data..."
data = list(parseData("http://jmcauley.ucsd.edu/cse190/data/beer/beer_50000.json"))
print "done"

Reading data...
done


In [6]:
# part 1 of the hw

tmp = [d['beer/beerId'] for d in data]
unique = set(tmp)
print "unique beerId is " , len(unique)

tmp = [d['user/profileName'] for d in data]
unique = set(tmp)
print "unique profileName is " , len(unique)

tmp = [d['review/appearance'] for d in data]
print "mean of review/appearance is ", np.mean(tmp)

tmp = [d['review/palate'] for d in data]
print "mean of review/apperance is ", np.mean(tmp)

tmp = [d['review/overall'] for d in data]
print "mean of review/overall is ", np.mean(tmp)

tmp = [d['review/aroma'] for d in data]
print "mean of review/aroma is ", np.mean(tmp)

tmp = [d['review/taste'] for d in data]
print "mean of review/taste is ", np.mean(tmp)

tmp = [d['beer/ABV'] for d in data]
print "mean of beer/ABV is ", np.mean(tmp)


unique beerId is  1923
unique profileName is  8376
mean of review/appearance is  3.8984
mean of review/apperance is  3.85267
mean of review/overall is  3.88871
mean of review/aroma is  3.87152
mean of review/taste is  3.92225
mean of beer/ABV is  7.400287


In [7]:
# part 2 of the hw

tmp = [d['review/taste'] for d in data]
print "varience of the revew/tast scores is ", np.var(tmp)

# actual review/taste
y_true = tmp
# predicting use mean value score
y_pred = [np.mean(tmp)]*len(tmp)
print "MSE is ",mean_squared_error(y_true,y_pred)

varience of the revew/tast scores is  0.5135699375
MSE is  0.5135699375


In [8]:
# part 3 train a linear regression predictor to use beer/ABV to predict review/taste
def feature(datum):
  feat = [1]
  feat.append(datum['beer/ABV'])
  return feat

X = [feature(d) for d in data]
y = [d['review/taste'] for d in data]
theta,residuals,rank,s = np.linalg.lstsq(X, y)
# the second theta is saying how much the function will change
print theta
# starting from 3.1152, for every 0.1 increase in ABV, the overall rating will increase by 0.10905507

[ 3.11521115  0.10905507]


In [9]:
#part 4
def split_list(a_list):
    half = len(a_list)/2
    return (a_list[:half], a_list[half:])

training,testing = split_list(data)
# #print data
# training = data[:len(data)/2]
# print len(training)
# testing = data[len(data)/2:]
# split the data into equal halfs,the fist half for training, and the second half for testing
#training,testing = split_list(data)
X = [feature(d) for d in training]
y = [d['review/taste'] for d in training]
theta,residuals,rank,s = np.linalg.lstsq(X, y)
print "theta values are ", theta

# predictor using the theta values - MSE for training
predictor = [np.dot(np.array(d),theta) for d in X]
print "MSE in training is ",mean_squared_error(y,predictor)

# predictor using the theta values - MSE for testing
X_test = [feature(d) for d in testing]
y_test = [d['review/taste'] for d in testing]
# predictor in the form Xi * theta
predictor = [np.dot(d,theta) for d in X_test]
print "MSE in testing is ",mean_squared_error(y_test,predictor)

theta values are  [ 2.99503282  0.11690802]
MSE in training is  0.483983105115
MSE in testing is  0.423776528023


In [10]:
# part 5 incorporate the time of day into predicting the taste rating
def feature(datum):
  feat = [1]
  #feat.append(datum['beer/ABV'])
  # length of the feature is 23
  tmp = [0]*23 
  if datum['review/timeStruct']['hour'] == 0:
        for i in tmp:
            feat.append(i)
  else:
    tmp[int(datum['review/timeStruct']['hour'])-1] = 1
    for i in tmp:
        feat.append(i)
  return feat

X = [feature(d) for d in training]
y = [d['review/taste'] for d in training]
theta,residuals,rank,s = np.linalg.lstsq(X, y)
print theta

X_test = [feature(d) for d in testing]
y_test = [d['review/taste'] for d in testing]
# predictor in the form Xi * theta
predictor = [np.dot(feature,theta) for feature in X_test]
print "MSE is ",mean_squared_error(y_test,predictor)

[  3.92582160e+00   3.97007042e-03   3.86265228e-02   4.42207313e-03
   4.97325107e-02  -8.22241008e-03  -2.02324724e-02  -1.03629083e-01
  -1.92081759e-01  -1.08828132e-01  -1.28435975e-01  -1.52862413e-01
  -6.39029916e-02  -1.43347369e-01  -4.80438185e-02  -2.42063246e-02
  -5.23709815e-03  -5.24520310e-02  -6.44730682e-02  -2.38329599e-02
  -4.27309814e-02  -3.19646756e-02  -3.64687779e-02  -1.61435509e-04]
MSE is  0.469519600621


In [11]:
# task 2 classifiction
print "Reading book description data ... "
data2 = list(parseData("http://jmcauley.ucsd.edu/cse190/data/amazon/book_descriptions_50000.json"))
print "done reading book description"

Reading book description data ... 
done reading book description


In [12]:
# part 1: calcualte the probablity of having categories Romance
prior = ["Romance" in b['categories'] for b in data2]
print "p (has category Romance) is ", float(sum(prior))/len(data2)
#print prior

p (has category Romance) is  0.03198


In [24]:
# part 2: p(mentions “love” in description | has category “Romance”)
prior = ['love' in b['description'] for b in data2]
# p(mentions love in description)
prior = sum(prior) * 1.0 / len(prior)
print "p(mentions love in description) is ", prior

# p(has category "Romance" | mentions "love" in description)
likelihood = ["Romance" in b['categories'] for b in data2 if 'love' in b['description']]
likelihood = sum(likelihood) * 1.0 / len(likelihood)
print "p(has category Romance | mentions love in description) is ", likelihood

# p(has category Romance)
print "p(has category Romance) is ", 0.03198

result = (prior*likelihood)/0.03198
print "final result is ", result

p(mentions love in description) is  0.17988
p(has category Romance | mentions love in description) is  0.0867244829887
p(has category Romance) is  0.03198
final result is  0.487804878049
True


In [31]:
# part 3 p(has category “Romance” | mentions “love” in description ∧ mentions “beaut” in description)
prior = ["Romance" in b['categories'] for b in data2 if 'love' in b['description'] and 'beaut' in b['description']]
prior = sum(prior)*1.0/len(prior)
print "nominator is ", prior

denominator = 1-prior
print "denominator is ", denominator
classfier1 = prior>denominator
print classfier1

# p(has category “Romance” | doesn't mentions “love” in description ∧ mentions “beaut” in description)
prior = ["Romance" in b['categories'] for b in data2 if not 'love' in b['description'] and 'beaut' in b['description']]
prior = sum(prior)*1.0/len(prior)
print "nominator is ", prior

denominator = 1-prior
print "denominator is ", denominator
classfier2 = prior>denominator
print classfier2

# p(has category “Romance” | doesn't mentions “love” in description ∧ doesn't mentions “beaut” in description)
prior = ["Romance" in b['categories'] for b in data2 if not 'love' in b['description'] and not 'beaut' in b['description']]
prior = sum(prior)*1.0/len(prior)
print "nominator is ", prior

denominator = 1-prior
print "denominator is ", denominator
classfier3 = prior>denominator
print classfier3

# p(has category “Romance” | mentions “love” in description ∧ doesn't mention “beaut” in description)
prior = ["Romance" in b['categories'] for b in data2 if'love' in b['description'] and not 'beaut' in b['description']]
prior = sum(prior)*1.0/len(prior)
print "nominator is ", prior

denominator = 1-prior
print "denominator is ", denominator
classfier4 = prior>denominator
print classfier4

nominator is  0.110849056604
denominator is  0.889150943396
False
nominator is  0.0352402745995
denominator is  0.9647597254
False
nominator is  0.0191133664769
denominator is  0.980886633523
False
nominator is  0.0792842595287
denominator is  0.920715740471
False


In [43]:
# part 3 Report the above probability for the four possible feature combinations (mentions “love” and “beaut”; mentions “love” and doesn’t mention “beaut” etc.).
# mentions love and not beaut
TP = 0 # label true and predict true, classfier always predict false
FP = 0 # label false and predict true, classfier always predict false
FN = 0
tmp = ["Romance" in b['categories'] for b in data2 if'love' in b['description'] and not 'beaut' in b['description']]
FN = FN + sum(tmp)
tmp = ["Romance" in b['categories'] for b in data2 if 'love' in b['description'] and 'beaut' in b['description']]
FN = FN + sum(tmp)
tmp = ["Romance" in b['categories'] for b in data2 if not 'love' in b['description'] and 'beaut' in b['description']]
FN = FN + sum(tmp)
tmp = ["Romance" in b['categories'] for b in data2 if not 'love' in b['description'] and not 'beaut' in b['description']]
FN = FN + sum(tmp)
print "FN is ", FN
TN = len(data2) - FN
print "TN is ", TN

TPR = float(TP)/(TP+FN)
print "True positive rate is ", TPR
TNR = float(TN)/(TN+FP)
print "True negative rate is ", TNR
FPR = float(FP)/(FP+TN)
print "False positive rate is ", FPR
FNR = float(FN)/(FN+TP)
print "False positive rate is ", FNR


FN is  1599
TN is  48401
True positive rate is  0.0
True negative rate is  1.0
False positive rate is  0.0
False positive rate is  1.0
